In [2]:
from qiskit import *
from qiskit.visualization import plot_distribution
import numpy as np, matplotlib.pyplot as plt

plt.style.use(['science', 'notebook', 'grid', 'dark_background'])

AttributeError: partially initialized module 'qiskit' has no attribute '_accelerate' (most likely due to a circular import)

*Maxwell's Demon* (ooh, spooky) is a thought experiment that was first proposed by James Clerk Maxwell in 1867. The idea is that if you had a demon that could open and close a door between two chambers of gas, you could use it to create a temperature difference between the two chambers **without doing any work,** thereby violating the Second Law of Thermodynamics.

A 'microscopic form' of this paradox is *Szilard's Engine,* which is a single-particle version of the demon. The particle can be in one of two states, and the demon can measure the state of the particle and use that information to extract work from the system.

Using quantum circuits, we can demonstrate the components of Szilard's Engine and show that it is possible to extract work from a single particle in a quantum state: